<a href="https://colab.research.google.com/github/lucacamus13/Tesis/blob/main/Limpieza_de_datos_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Primero instalar las librerías necesarias
!pip install pandas numpy yfinance fredapi matplotlib seaborn

FRED API KEY: 73be39590b688a9ca91a02451eed4cc5

# Obtención de variables macro

-FEDFUNDS: mensual, tasa porcentual anual, representa la tasa de interés promedio a la que las instituciones prestan fondos entre si durante la noche.

-PCEPI: mensual, índice, mide los cambios en los precios de bienes y servicios consumidos por hogares.

-INDPRO: mensual, índice, mide volumen de producción de la industria.

-RBUSBIS: mensual, índice, mide el valor promedio del dólar respecto a una canasta de otras monedas.

In [ ]:
import pandas as pd
from fredapi import Fred
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')

# Configuración de Estilo
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

def extract_fred_series(api_key='73be39590b688a9ca91a02451eed4cc5'):
    """
    Extraer series específicas de FRED y combinarlas en un DataFrame.
    Se ha eliminado FEDFUNDS de esta lista.
    """
    print("📊 EXTRAYENDO SERIES DE FRED")
    print("="*40)

    # Inicializar FRED API
    fred = Fred(api_key=api_key)

    # Series a extraer (FEDFUNDS ELIMINADO)
    series_codes = {
        'PCEPI': 'PCE Price Index',
        'INDPRO': 'Industrial Production Index',
        'RBUSBIS': 'Real Broad US Dollar Index'
    }

    # DataFrame para almacenar todas las series
    df = pd.DataFrame()

    # Extraer cada serie
    for code, description in series_codes.items():
        try:
            print(f"📈 Extrayendo {code} - {description}...")
            series = fred.get_series(code)

            if not series.empty:
                df[code] = series
                print(f"   ✅ {len(series)} observaciones desde {series.index.min().date()} hasta {series.index.max().date()}")
            else:
                print(f"   ⚠️ Sin datos para {code}")

        except Exception as e:
            print(f"   ❌ Error con {code}: {e}")

    # Resamplear a inicio de mes para asegurar consistencia
    if not df.empty:
        df = df.resample('MS').first()

    return df

def process_ff4_shock():
    """
    Procesar la variable de Shock FF4 desde GitHub
    Fuente: Acosta (2022) / GSS
    Hoja: "Monetary Events"
    Columnas: B (date_time), K (FF4)
    Metodología: Suma mensual de eventos intradía. Rellenar NaNs con 0.
    """
    print("\n⚡ PROCESANDO SHOCK DE POLÍTICA MONETARIA (FF4)")
    print("="*40)

    # URL Raw para descarga directa
    url = "https://github.com/lucacamus13/Tesis/raw/main/data/USMPD-FF4%20EEUU.xlsx"

    try:
        print(f"📥 Descargando datos desde: {url}")
        # Leer archivo Excel especificando la hoja
        # Se asume que la primera fila contiene los encabezados
        df_ff4 = pd.read_excel(url, sheet_name='Monetary Events')

        # Normalizar nombres de columnas (minusculas y sin espacios extra)
        df_ff4.columns = [str(col).strip() for col in df_ff4.columns]
        print(f"   Columnas encontradas: {df_ff4.columns.tolist()}")

        # Identificar columnas específicas
        # Buscamos 'date_time' (o similar) y 'FF4'
        date_col = None
        ff4_col = None

        for col in df_ff4.columns:
            if 'date_time' in col.lower():
                date_col = col
            if 'ff4' in col.lower() and 'ff4' == col.split()[0].lower(): # Tratar de coincidir exacto o prefijo
                ff4_col = col

        # Fallback si no encuentra exacto, buscar por posición si el usuario indicó B y K
        # B es index 1, K es index 10 (0-indexed)
        if date_col is None and len(df_ff4.columns) > 1:
             date_col = df_ff4.columns[1]
             print(f"   ⚠️ No se encontró columna 'date_time', usando columna índice 1: {date_col}")

        if ff4_col is None:
            # Buscar cualquier columna que contenga FF4
            candidates = [c for c in df_ff4.columns if 'ff4' in c.lower()]
            if candidates:
                ff4_col = candidates[0]
            elif len(df_ff4.columns) > 10:
                ff4_col = df_ff4.columns[10]
                print(f"   ⚠️ No se encontró columna 'FF4', usando columna índice 10: {ff4_col}")

        if date_col is None or ff4_col is None:
            raise ValueError(f"No se pudieron identificar las columnas. Date: {date_col}, FF4: {ff4_col}")

        print(f"   📅 Columna Fecha: {date_col}")
        print(f"   🎯 Columna Datos: {ff4_col}")

        # Convertir fecha
        df_ff4[date_col] = pd.to_datetime(df_ff4[date_col])
        df_ff4 = df_ff4.set_index(date_col)

        # Seleccionar solo la serie necesaria y asegurar numérico
        shock_series = df_ff4[[ff4_col]].copy()
        shock_series[ff4_col] = pd.to_numeric(shock_series[ff4_col], errors='coerce')
        shock_series.columns = ['FF4_Daily']

        # Agrupación Mensual (Suma de eventos)
        # Nota: El usuario indicó que son datos intradía/diarios y quiere "mensualizar sumando"
        print("   🔄 Agrupando datos a frecuencia MENSUAL (Suma)...")
        shock_monthly = shock_series.resample('MS').sum()

        # Rellenar meses sin datos con 0.0 (según metodología)
        # Crear índice completo
        full_idx = pd.date_range(start=shock_monthly.index.min(),
                               end=shock_monthly.index.max(),
                               freq='MS')

        shock_final = shock_monthly.reindex(full_idx).fillna(0.0)
        shock_final.columns = ['FF4']

        print(f"   ✅ Procesamiento completado. {len(shock_final)} observaciones.")
        print(f"   ℹ️ Meses con shock != 0: {(shock_final['FF4'] != 0).sum()}")
        print(f"   ℹ️ Rango: {shock_final.index.min().date()} - {shock_final.index.max().date()}")

        return shock_final

    except Exception as e:
        print(f"   ❌ Error procesando FF4: {e}")
        # Imprimir columnas disponibles para debug
        try:
            debug_df = pd.read_excel(url, sheet_name='Monetary Events', nrows=5)
            print(f"   ℹ️ Primeras columnas del archivo: {debug_df.columns.tolist()}")
        except:
            pass
        return None

def main():
    print("🚀 INICIANDO LIMPIEZA DE DATOS")

    # 1. Datos Macro FRED
    macro_df = extract_fred_series()

    # 2. Datos Shock FF4
    ff4_df = process_ff4_shock()

    # Combinar
    if not macro_df.empty and ff4_df is not None:
        print("\n🔗 COMBINANDO BASES DE DATOS...")
        # Merge por índice (fecha)
        full_df = macro_df.join(ff4_df, how='outer')

        # Recortar al rango donde empiezan los datos de FF4
        start_date = ff4_df.index.min()
        full_df = full_df.loc[start_date:]

        print(f"   ✅ Base consolidada: {full_df.shape}")

        # Mostrar últimas filas
        print("\n📋 ÚLTIMAS 5 OBSERVACIONES:")
        print(full_df.tail())

        return full_df
    else:
        print("\n⚠️ No se pudo combinar las bases.")
        return None

if __name__ == "__main__":
    df_final = main()


🚀 INICIANDO LIMPIEZA DE DATOS
📊 EXTRAYENDO SERIES DE FRED
📈 Extrayendo PCEPI - PCE Price Index...
   ✅ 801 observaciones desde 1959-01-01 hasta 2025-09-01
📈 Extrayendo INDPRO - Industrial Production Index...
   ✅ 1281 observaciones desde 1919-01-01 hasta 2025-09-01
📈 Extrayendo RBUSBIS - Real Broad US Dollar Index...
   ✅ 383 observaciones desde 1994-01-01 hasta 2025-11-01

⚡ PROCESANDO SHOCK DE POLÍTICA MONETARIA (FF4)
📥 Descargando datos desde: https://github.com/lucacamus13/Tesis/raw/main/data/USMPD-FF4%20EEUU.xlsx
   Columnas encontradas: ['Date', 'date_time', 'SEP', 'Unscheduled', 'PC', 'MP1', 'MP2', 'FF1', 'FF2', 'FF3', 'FF4', 'FF5', 'FF6', 'ED1', 'ED2', 'ED3', 'ED4', 'ED5', 'ED6', 'ED7', 'ED8', 'OIS1Y', 'OIS2Y', 'UST3M', 'UST6M', 'UST2Y', 'UST5Y', 'UST10Y', 'UST30Y', 'TIPS5Y', 'TIPS10Y', 'TIPS30Y', 'SP500', 'SPFUT', 'DXY', 'EURUSD', 'USDJPY']
   📅 Columna Fecha: date_time
   🎯 Columna Datos: FF4
   🔄 Agrupando datos a frecuencia MENSUAL (Suma)...
   ✅ Procesamiento completado. 3

In [ ]:
import pandas as pd
from fredapi import Fred
import warnings
warnings.filterwarnings('ignore')

def extract_fred_series(api_key='73be39590b688a9ca91a02451eed4cc5'):
    """
    Extraer series específicas de FRED y combinarlas en un DataFrame
    """
    print("📊 EXTRAYENDO SERIES DE FRED")
    print("="*40)

    # Inicializar FRED API
    fred = Fred(api_key=api_key)

    # Series a extraer
    series_codes = {
        'FEDFUNDS': 'Federal Funds Rate',
        'PCEPI': 'PCE Price Index',
        'INDPRO': 'Industrial Production Index',
        'RBUSBIS': 'Real Broad US Dollar Index'
    }

    # DataFrame para almacenar todas las series
    df = pd.DataFrame()

    # Extraer cada serie
    for code, description in series_codes.items():
        try:
            print(f"📈 Extrayendo {code} - {description}...")

            # Obtener toda la data histórica disponible
            series = fred.get_series(code)

            if not series.empty:
                df[code] = series
                print(f"   ✅ {len(series)} observaciones desde {series.index.min()} hasta {series.index.max()}")
            else:
                print(f"   ⚠️ Sin datos para {code}")

        except Exception as e:
            print(f"   ❌ Error con {code}: {e}")

    # Información del DataFrame combinado
    print("\n📋 DATAFRAME COMBINADO:")
    print(f"   - Variables: {len(df.columns)}")
    print(f"   - Observaciones totales: {len(df)}")

    if not df.empty:
        print(f"   - Rango temporal: {df.index.min()} a {df.index.max()}")
        print(f"   - Observaciones válidas después de combinar: {len(df.dropna())}")

    # Mostrar información de cada serie
    print("\n📊 INFORMACIÓN POR SERIE:")
    for col in df.columns:
        valid_data = df[col].dropna()
        print(f"   {col:<10}: {len(valid_data)} obs válidas "
              f"({valid_data.index.min()} a {valid_data.index.max()})")

    # Estadísticas básicas
    if not df.empty:
        print("\n📈 ESTADÍSTICAS BÁSICAS:")
        print(df.describe().round(3))

    return df


# Función principal
def main():
    """
    Ejecutar extracción completa
    """
    print("🚀 INICIANDO EXTRACCIÓN DE DATOS FRED")
    print("API Key configurada ✅\n")

    # Extraer datos
    combined_df = extract_fred_series()

    if not combined_df.empty:
        print("\n🎉 ¡EXTRACCIÓN COMPLETADA EXITOSAMENTE!")
        print(f"📊 DataFrame final: {combined_df.shape[0]} filas × {combined_df.shape[1]} columnas")

        return combined_df
    else:
        print("\n❌ No se pudieron extraer los datos")
        return None

# Ejecutar
if __name__ == "__main__":
    result_df = main()

    # Mostrar primeras filas si hay datos
    if result_df is not None and not result_df.empty:
        print("\n📋 PRIMERAS 10 OBSERVACIONES:")
        print(result_df.head(10).round(3))

🚀 INICIANDO EXTRACCIÓN DE DATOS FRED
API Key configurada ✅

📊 EXTRAYENDO SERIES DE FRED
📈 Extrayendo FEDFUNDS - Federal Funds Rate...
   ✅ 855 observaciones desde 1954-07-01 00:00:00 hasta 2025-09-01 00:00:00
📈 Extrayendo PCEPI - PCE Price Index...
   ✅ 800 observaciones desde 1959-01-01 00:00:00 hasta 2025-08-01 00:00:00
📈 Extrayendo INDPRO - Industrial Production Index...
   ✅ 1280 observaciones desde 1919-01-01 00:00:00 hasta 2025-08-01 00:00:00
📈 Extrayendo RBUSBIS - Real Broad US Dollar Index...
   ✅ 380 observaciones desde 1994-01-01 00:00:00 hasta 2025-08-01 00:00:00

📋 DATAFRAME COMBINADO:
   - Variables: 4
   - Observaciones totales: 855
   - Rango temporal: 1954-07-01 00:00:00 a 2025-09-01 00:00:00
   - Observaciones válidas después de combinar: 380

📊 INFORMACIÓN POR SERIE:
   FEDFUNDS  : 855 obs válidas (1954-07-01 00:00:00 a 2025-09-01 00:00:00)
   PCEPI     : 800 obs válidas (1959-01-01 00:00:00 a 2025-08-01 00:00:00)
   INDPRO    : 854 obs válidas (1954-07-01 00:00:00 a 

In [ ]:
# Asegurar que el índice sea de tipo datetime y tenga frecuencia mensual
combined_df_monthly = result_df.resample('MS').first()

# Encontrar el rango de fechas donde todas las series tienen datos (después del resample)
start_date = combined_df_monthly.first_valid_index()
end_date = combined_df_monthly.last_valid_index()

# Filtrar el DataFrame por este rango de fechas común
fixed_macro_data = combined_df_monthly.loc[start_date:end_date]

# Opcional: Rellenar NaNs si es necesario (por ejemplo, con el valor anterior o una interpolación)
# Aquí usamos forward fill para los NaNs que puedan quedar dentro del rango común
fixed_macro_data = fixed_macro_data.fillna(method='ffill').fillna(method='bfill')

print("DataFrame combinado con rango de fechas común:")
display(fixed_macro_data.head())
display(fixed_macro_data.tail())
print(f"Shape del DataFrame final: {fixed_macro_data.shape}")

DataFrame combinado con rango de fechas común:


,FEDFUNDS,PCEPI,INDPRO,RBUSBIS
1954-07-01,0.80,15.164,18.0907,88.69
1954-08-01,1.22,15.164,18.0639,88.69
1954-09-01,1.07,15.164,18.0908,88.69
1954-10-01,0.85,15.164,18.3058,88.69
1954-11-01,0.83,15.164,18.6015,88.69


,FEDFUNDS,PCEPI,INDPRO,RBUSBIS
2025-05-01,4.33,126.380,103.6570,109.90
2025-06-01,4.33,126.743,104.2115,108.71
2025-07-01,4.33,126.949,103.8194,108.04
2025-08-01,4.33,127.285,103.9203,108.68
2025-09-01,4.22,127.285,103.9203,108.68


Shape del DataFrame final: (855, 4)


Codigo para exportar a Excel el df

In [ ]:
# Exportar el DataFrame fixed_macro_data a un archivo de Excel
excel_filename = 'macro_data_cleaned.xlsx'
fixed_macro_data.to_excel(excel_filename)

print(f"DataFrame 'fixed_macro_data' exportado a '{excel_filename}' exitosamente.")

DataFrame 'fixed_macro_data' exportado a 'macro_data_cleaned.xlsx' exitosamente.


# Obtención de datos financieros

-S&P500: diario a mensual, último dato del mes. Datos desde 1950

-VIX: diario a mensual, promedio mensual. Datos desde 1990

In [ ]:
import pandas as pd
import yfinance as yf
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

def extract_sp500_vix_simple():
    """
    Extraer solo S&P 500 y VIX desde Yahoo Finance
    Sin variables adicionales - solo las series básicas
    """
    print("📊 EXTRAYENDO S&P 500 Y VIX - SERIES BÁSICAS")
    print("="*45)

    # DataFrame para almacenar datos
    df = pd.DataFrame()

    # ==========================================
    # 1. S&P 500 (^GSPC)
    # ==========================================
    print("\n📈 EXTRAYENDO S&P 500 (^GSPC)...")

    try:
        # Extraer desde 1950 para obtener máxima historia
        sp500_data = yf.download('^GSPC', start='1950-01-01', end=datetime.now().strftime('%Y-%m-%d'), progress=False)

        if not sp500_data.empty:
            # Usar precio de cierre ajustado
            sp500_daily = sp500_data['Close']

            # Convertir a mensual (último valor del mes)
            sp500_monthly = sp500_daily.resample('M').last()

            df['SP500'] = sp500_monthly

            print(f"   ✅ S&P 500 extraído exitosamente")
            print(f"   📊 {len(sp500_monthly)} observaciones mensuales")
            print(f"   📅 Desde: {sp500_monthly.index.min().strftime('%Y-%m-%d')}")
            print(f"   📅 Hasta: {sp500_monthly.index.max().strftime('%Y-%m-%d')}")

        else:
            print("   ❌ No se pudieron obtener datos del S&P 500")

    except Exception as e:
        print(f"   ❌ Error extrayendo S&P 500: {e}")

    # ==========================================
    # 2. VIX (^VIX)
    # ==========================================
    print("\n📊 EXTRAYENDO VIX (^VIX)...")

    try:
        # Extraer desde 1985 para obtener máxima historia del VIX
        vix_data = yf.download('^VIX', start='1985-01-01', end=datetime.now().strftime('%Y-%m-%d'), progress=False)

        if not vix_data.empty:
            # Usar precio de cierre
            vix_daily = vix_data['Close']

            # Convertir a mensual (promedio del mes)
            vix_monthly = vix_daily.resample('M').mean()

            df['VIX'] = vix_monthly

            print(f"   ✅ VIX extraído exitosamente")
            print(f"   📊 {len(vix_monthly)} observaciones mensuales")
            print(f"   📅 Desde: {vix_monthly.index.min().strftime('%Y-%m-%d')}")
            print(f"   📅 Hasta: {vix_monthly.index.max().strftime('%Y-%m-%d')}")

        else:
            print("   ❌ No se pudieron obtener datos del VIX")

    except Exception as e:
        print(f"   ❌ Error extrayendo VIX: {e}")

    # ==========================================
    # 3. RESUMEN FINAL
    # ==========================================
    print("\n📋 DATASET FINAL:")
    print(f"   - Variables: {list(df.columns)}")
    print(f"   - Observaciones totales: {len(df)}")

    if not df.empty:
        print(f"   - Período completo: {df.index.min().strftime('%Y-%m')} a {df.index.max().strftime('%Y-%m')}")

        # Información por variable
        for col in df.columns:
            valid_data = df[col].dropna()
            if not valid_data.empty:
                print(f"   - {col}: {len(valid_data)} observaciones válidas")

        # Período común
        common_data = df.dropna()
        if not common_data.empty:
            print(f"   - Período común: {len(common_data)} observaciones")
            print(f"   - Desde: {common_data.index.min().strftime('%Y-%m')}")
            print(f"   - Hasta: {common_data.index.max().strftime('%Y-%m')}")

        # Estadísticas básicas
        print("\n📈 ESTADÍSTICAS BÁSICAS:")
        print(df.describe().round(2))

    return df

def export_data(df, filename='sp500_vix_simple.csv'):
    """
    Exportar datos a CSV
    """
    if not df.empty:
        df.to_csv(filename)
        print(f"\n💾 Datos exportados a: {filename}")
        return True
    else:
        print("\n❌ No hay datos para exportar")
        return False

def main():
    """
    Ejecutar extracción simple
    """
    print("🚀 EXTRACCIÓN SIMPLE: S&P 500 Y VIX")
    print("Solo series básicas, sin transformaciones")
    print("="*45)

    # Extraer datos
    df = extract_sp500_vix_simple()

    if not df.empty:
        # Exportar
        export_data(df)

        print("\n🎉 ¡EXTRACCIÓN COMPLETADA!")
        print(f"📊 {df.shape[0]} filas × {df.shape[1]} columnas")

        return df
    else:
        print("\n❌ No se pudieron extraer los datos")
        return None

# Ejecutar
if __name__ == "__main__":
    result_df = main()

    # Mostrar muestra
    if result_df is not None and not result_df.empty:
        print("\n📋 MUESTRA DE DATOS:")
        print(result_df.head(10).round(2))
        print("\n...")
        print(result_df.tail(5).round(2))

🚀 EXTRACCIÓN SIMPLE: S&P 500 Y VIX
Solo series básicas, sin transformaciones
📊 EXTRAYENDO S&P 500 Y VIX - SERIES BÁSICAS

📈 EXTRAYENDO S&P 500 (^GSPC)...
   ✅ S&P 500 extraído exitosamente
   📊 909 observaciones mensuales
   📅 Desde: 1950-01-31
   📅 Hasta: 2025-09-30

📊 EXTRAYENDO VIX (^VIX)...
   ✅ VIX extraído exitosamente
   📊 429 observaciones mensuales
   📅 Desde: 1990-01-31
   📅 Hasta: 2025-09-30

📋 DATASET FINAL:
   - Variables: ['SP500', 'VIX']
   - Observaciones totales: 909
   - Período completo: 1950-01 a 2025-09
   - SP500: 909 observaciones válidas
   - VIX: 429 observaciones válidas
   - Período común: 429 observaciones
   - Desde: 1990-01
   - Hasta: 2025-09

📈 ESTADÍSTICAS BÁSICAS:
         SP500     VIX
count   909.00  429.00
mean    899.76   19.47
std    1277.60    7.44
min      17.05   10.13
25%      89.25   14.08
50%     272.02   17.68
75%    1267.38   23.08
max    6688.46   62.64

💾 Datos exportados a: sp500_vix_simple.csv

🎉 ¡EXTRACCIÓN COMPLETADA!
📊 909 filas × 2

In [ ]:
# Asegurar que el índice sea de tipo datetime y tenga frecuencia mensual
# Aunque yfinance ya devuelve un DatetimeIndex, resample('MS') lo asegura y establece la frecuencia
sp500_vix_monthly = result_df.resample('MS').first()

# Encontrar el rango de fechas donde ambas series tienen datos (después del resample)
# Usamos dropna() para encontrar el primer y último índice donde *ambas* columnas tienen datos
common_period_df = sp500_vix_monthly.dropna()

if not common_period_df.empty:
    start_date_financial = common_period_df.index.min()
    end_date_financial = common_period_df.index.max()

    # Filtrar el DataFrame original resampleado por este rango de fechas común
    fixed_financial_data = sp500_vix_monthly.loc[start_date_financial:end_date_financial]

    # Opcional: Rellenar NaNs si es necesario dentro del rango común (menos probable aquí)
    # fixed_financial_data = fixed_financial_data.fillna(method='ffill').fillna(method='bfill')

    print("DataFrame financiero combinado con rango de fechas común:")
    display(fixed_financial_data.head())
    display(fixed_financial_data.tail())
    print(f"Shape del DataFrame final: {fixed_financial_data.shape}")
else:
    fixed_financial_data = pd.DataFrame()
    print("No se encontró un período de fechas común con datos válidos para ambas series.")

DataFrame financiero combinado con rango de fechas común:


,SP500,VIX
Date,,
1990-01-01,329.079987,23.347273
1990-02-01,331.890015,23.262632
1990-03-01,339.940002,20.062273
1990-04-01,330.799988,21.403500
1990-05-01,361.230011,18.097727


,SP500,VIX
Date,,
2025-05-01,5911.689941,20.457143
2025-06-01,6204.950195,18.215000
2025-07-01,6339.390137,16.331363
2025-08-01,6460.259766,15.750000
2025-09-01,6688.459961,15.773333


Shape del DataFrame final: (429, 2)


# Economías Emergentes